# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("AZURE_OPENAI_API_KEY")
_set_env("AZURE_OPENAI_ENDPOINT")
_set_env("AZURE_OPENAI_API_VERSION")
_set_env("AZURE_OPENAI_DEPLOYMENT_NAME")

In [4]:
llm_config: dict = {
    "config_list": [
        {
            "model": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            "api_type": "azure",
            "api_key": os.environ["AZURE_OPENAI_API_KEY"],
            "base_url": os.environ["AZURE_OPENAI_ENDPOINT"],
            "api_version": os.environ["AZURE_OPENAI_API_VERSION"],
        }
    ]
}

## Creating the needed agents

In [17]:
from autogen import ConversableAgent

In [18]:
onboarding_personal_information_agent: ConversableAgent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message="""You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [19]:
onboarding_topic_preference_agent: ConversableAgent = ConversableAgent(
    name="Onboarding_Topic_preference_Agent",
    system_message="""You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [20]:
customer_engagement_agent: ConversableAgent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message="""You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [21]:
customer_proxy_agent: ConversableAgent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [ ]:
chats: list[dict] = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product."
        "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information "
            "into as JSON object only: "
            "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history": True,
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are "
        "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False,
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [23]:
from autogen import initiate_chats
from autogen.agentchat.chat import ChatResult

chat_results: list[ChatResult] = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Alex

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Thank you, Alex! And could you please tell me your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Modena

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (dae81c08-541d-4126-af

## Print out the summary

In [24]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "name": "Alex",
  "location": "Modena"
}
```


Alex, located in Modena, is interested in reading about F1.


Alex is interested in Formula 1 and enjoys learning fun facts about it, especially considering their location in Modena, an automotive hub. Key points shared include Modena's significance in motorsport, the fastest pit stop record, historical relevance of F1 in Italy, and the drama of team orders. A light-hearted F1 joke was also included for amusement.




## Print out the cost

In [25]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 5.4449999999999995e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.4449999999999995e-05, 'prompt_tokens': 215, 'completion_tokens': 37, 'total_tokens': 252}}, 'usage_excluding_cached_inference': {'total_cost': 5.4449999999999995e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.4449999999999995e-05, 'prompt_tokens': 215, 'completion_tokens': 37, 'total_tokens': 252}}}


{'usage_including_cached_inference': {'total_cost': 2.2349999999999998e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.2349999999999998e-05, 'prompt_tokens': 85, 'completion_tokens': 16, 'total_tokens': 101}}, 'usage_excluding_cached_inference': {'total_cost': 2.2349999999999998e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.2349999999999998e-05, 'prompt_tokens': 85, 'completion_tokens': 16, 'total_tokens': 101}}}


{'usage_including_cached_inference': {'total_cost': 0.00033749999999999996, 'gpt-4o-mini-2024-07-18': {'cost': 0.00033749999999999996, 'prompt_tokens': 562, 'completion_tokens': 42